In [1]:
import json
from collections import defaultdict
from collections import OrderedDict

In [2]:
with open("../Observed_data/LTT_observed.txt") as json_file:
    LTT = json.load(json_file)

In [3]:
print(len(LTT))

370


In [13]:
#So this function forces simulated LTT into the same bins as the observed

def bin_sim(sim_ltt, obs_bins):
    
    intervals = []
    lineages = []
    bin_dict = defaultdict(list)
    sim_bins = {}



    for key, value in sim_ltt.items():
        start_interval = key[0]
        end_interval = key[1]
        lineage_number = value
        

        intervals.append((start_interval, end_interval))
        lineages.append(lineage_number)    

        
    

    for position, interval in enumerate(intervals):

        for bin_position,binn in enumerate(obs_bins):

            if interval[0] == 0 and binn[0] == 0.0:
                bin_dict[bin_position].append(position)

            if (interval[0] > binn[0] and interval[0] <= binn[1]) or (interval[1] > binn[0] and interval[1] <= binn[1]) or (interval[0] < binn[0] and interval[1] > binn[1]):

                bin_dict[bin_position].append(position)

        

    for binn_pos, interval_pos_list in bin_dict.items():

        if len(interval_pos_list) == 1:

            sim_bins[obs_bins[binn_pos]] = lineages[interval_pos_list[0]]


        if len(interval_pos_list) == 2:

            lineage_num_a = lineages[interval_pos_list[0]]
            lineage_num_b  = lineages[interval_pos_list[1]]

            interval_a = intervals[interval_pos_list[0]][1]

            bin_a = obs_bins[binn_pos][0]
            bin_b = obs_bins[binn_pos][1]


            weight_a = (interval_a - bin_a)
            weight_b = 1 - weight_a

            lineage_num = ((lineage_num_a*weight_a) + (lineage_num_b*weight_b))

            sim_bins[obs_bins[binn_pos]] = lineage_num


        elif len(interval_pos_list) > 2:

            last_position = len(interval_pos_list) - 1

            lineage_num_a = lineages[interval_pos_list[0]]
            lineage_num_b = lineages[interval_pos_list[last_position]]

            other_lins = [lineages[interval_pos_list[i]] for i in range(1, len(interval_pos_list)-1)]

            interval_a = intervals[interval_pos_list[0]][1]
            interval_b = intervals[interval_pos_list[last_position]][0]        

            bin_a = obs_bins[binn_pos][0]
            bin_b = obs_bins[binn_pos][1]

            weight_a = (interval_a - bin_a)
            weight_b = (bin_b - interval_b)

            other_weights = (1 - weight_a - weight_b)/(len(interval_pos_list)-2)

            final_other_lins = 0

            for i in other_lins:
                final_other_lins += (other_weights*i)

            lins = ((lineage_num_a*weight_a) + (lineage_num_b*weight_b) + final_other_lins)

            sim_bins[obs_bins[binn_pos]] = lins
            
            
            
    return sim_bins

In [5]:
new_LTT = {}

for key, value in LTT.items():
    a = float(key.split(",")[0].strip("("))
    b = float(key.split(",")[1].strip(")").strip(" "))
    
    new_LTT[a,b] = value

In [6]:
coal_points = set()
for key in new_LTT.keys():
    for i in key:
        coal_points.add(i)

In [7]:
len(coal_points)

371

In [8]:
biggest = max(coal_points)
print(biggest)

0.4272233755511325


In [9]:
interval_size = biggest/20
print(interval_size)

0.021361168777556623


In [10]:
count = 0
intervals = []
for i in range(20):
    
    new_tup = (count, count+interval_size)
    print(new_tup)
    count += interval_size
    
    intervals.append(new_tup)

(0, 0.021361168777556623)
(0.021361168777556623, 0.042722337555113246)
(0.042722337555113246, 0.06408350633266988)
(0.06408350633266988, 0.08544467511022649)
(0.08544467511022649, 0.10680584388778311)
(0.10680584388778311, 0.12816701266533972)
(0.12816701266533972, 0.14952818144289634)
(0.14952818144289634, 0.17088935022045296)
(0.17088935022045296, 0.19225051899800957)
(0.19225051899800957, 0.2136116877755662)
(0.2136116877755662, 0.2349728565531228)
(0.2349728565531228, 0.25633402533067945)
(0.25633402533067945, 0.27769519410823607)
(0.27769519410823607, 0.2990563628857927)
(0.2990563628857927, 0.3204175316633493)
(0.3204175316633493, 0.3417787004409059)
(0.3417787004409059, 0.36313986921846253)
(0.36313986921846253, 0.38450103799601915)
(0.38450103799601915, 0.40586220677357576)
(0.40586220677357576, 0.4272233755511324)


In [11]:
print(len(intervals))

20


In [14]:
binned_obs_LTT = bin_sim(new_LTT, intervals)

In [15]:
binned_obs_LTT

{(0, 0.021361168777556623): 59.64044861194819,
 (0.021361168777556623, 0.042722337555113246): 124.75396017627713,
 (0.042722337555113246, 0.06408350633266988): 133.9364509309589,
 (0.06408350633266988, 0.08544467511022649): 148.97678349186248,
 (0.08544467511022649, 0.10680584388778311): 148.33907671517255,
 (0.10680584388778311, 0.12816701266533972): 91.59093272776481,
 (0.12816701266533972, 0.14952818144289634): 70.07876927609597,
 (0.14952818144289634, 0.17088935022045296): 58.46111323028046,
 (0.17088935022045296, 0.19225051899800957): 50.24916890609602,
 (0.19225051899800957, 0.2136116877755662): 55.63545914760437,
 (0.2136116877755662, 0.2349728565531228): 72.6885652880981,
 (0.2349728565531228, 0.25633402533067945): 67.61114496403488,
 (0.25633402533067945, 0.27769519410823607): 41.31670189441371,
 (0.27769519410823607, 0.2990563628857927): 48.50878216596301,
 (0.2990563628857927, 0.3204175316633493): 40.355851127800555,
 (0.3204175316633493, 0.3417787004409059): 27.025321871739

In [16]:
LTT_list = []

for i in binned_obs_LTT.values():
    LTT_list.append(i)

In [17]:
print(LTT_list)

[59.64044861194819, 124.75396017627713, 133.9364509309589, 148.97678349186248, 148.33907671517255, 91.59093272776481, 70.07876927609597, 58.46111323028046, 50.24916890609602, 55.63545914760437, 72.6885652880981, 67.61114496403488, 41.31670189441371, 48.50878216596301, 40.355851127800555, 27.025321871739628, 19.004614160153153, 11.476947041818141, 2.0021308852811104, 2]


In [18]:
print(intervals)

[(0, 0.021361168777556623), (0.021361168777556623, 0.042722337555113246), (0.042722337555113246, 0.06408350633266988), (0.06408350633266988, 0.08544467511022649), (0.08544467511022649, 0.10680584388778311), (0.10680584388778311, 0.12816701266533972), (0.12816701266533972, 0.14952818144289634), (0.14952818144289634, 0.17088935022045296), (0.17088935022045296, 0.19225051899800957), (0.19225051899800957, 0.2136116877755662), (0.2136116877755662, 0.2349728565531228), (0.2349728565531228, 0.25633402533067945), (0.25633402533067945, 0.27769519410823607), (0.27769519410823607, 0.2990563628857927), (0.2990563628857927, 0.3204175316633493), (0.3204175316633493, 0.3417787004409059), (0.3417787004409059, 0.36313986921846253), (0.36313986921846253, 0.38450103799601915), (0.38450103799601915, 0.40586220677357576), (0.40586220677357576, 0.4272233755511324)]


## Getting the SS's

In [19]:
ltt_dict = OrderedDict(binned_obs_LTT)

max_L = max(ltt_dict.values())

In [20]:
for k,v in ltt_dict.items():
    print(k,v)

(0, 0.021361168777556623) 59.64044861194819
(0.021361168777556623, 0.042722337555113246) 124.75396017627713
(0.042722337555113246, 0.06408350633266988) 133.9364509309589
(0.06408350633266988, 0.08544467511022649) 148.97678349186248
(0.08544467511022649, 0.10680584388778311) 148.33907671517255
(0.10680584388778311, 0.12816701266533972) 91.59093272776481
(0.12816701266533972, 0.14952818144289634) 70.07876927609597
(0.14952818144289634, 0.17088935022045296) 58.46111323028046
(0.17088935022045296, 0.19225051899800957) 50.24916890609602
(0.19225051899800957, 0.2136116877755662) 55.63545914760437
(0.2136116877755662, 0.2349728565531228) 72.6885652880981
(0.2349728565531228, 0.25633402533067945) 67.61114496403488
(0.25633402533067945, 0.27769519410823607) 41.31670189441371
(0.27769519410823607, 0.2990563628857927) 48.50878216596301
(0.2990563628857927, 0.3204175316633493) 40.355851127800555
(0.3204175316633493, 0.3417787004409059) 27.025321871739628
(0.3417787004409059, 0.36313986921846253) 1

In [21]:
max_L

148.97678349186248

In [22]:
t_max_L = max(ltt_dict, key = lambda k:ltt_dict[k])[0]

In [23]:
t_max_L

0.06408350633266988

In [24]:
y1 = max_L
y0 = next(iter(ltt_dict.values()))

x1 = t_max_L
x0 = 0.0

In [25]:
slope_2 = abs((y1-y0)/(x1-x0)) #because we're going backwards and they're going forwards

In [26]:
slope_2

1394.061280232578

In [27]:
y2 = next(reversed(ltt_dict.values()))

x2 = next(reversed(ltt_dict.keys()))[1]


print(y2, x2)

2 0.4272233755511324


In [28]:
slope_1 = abs((y2-y1)/(x2-x1))

In [29]:
slope_1

404.73876858572703

In [30]:
slope_ratio = slope_1/slope_2

In [31]:
slope_ratio

0.29033068655217403

In [ ]:
from 